# AWS IoT Authentication and Authorization


Create a Thing, a unique name helps to make references more clear.
Any computer can be a thing, lets use this notebook instance as our thing to begin.

In [1]:
from datetime import datetime
HM = datetime.now().strftime('%H%M')
%store HM

Stored 'HM' (str)


In [2]:
thing_name = f"NotebookThing{HM}"
thing_name

'NotebookThing1035'

In [3]:
! aws iot create-thing --thing-name {thing_name}

{
    "thingName": "NotebookThing1035",
    "thingArn": "arn:aws:iot:us-east-1:030555009967:thing/NotebookThing1035",
    "thingId": "6bf2f7e2-e6bd-4ea3-8b4a-98721b8b3fa5"
}


Create a certificate and keys to authenticate your thing. 

In [4]:
from pathlib import Path
home = str(Path.home())
! mkdir "{home}/aws_iot"

certificate_file = f"{home}/aws_iot/certificate{HM}.pem"
public_key = f"{home}/aws_iot/public-key{HM}.pem"
private_key = f"{home}/aws_iot/private-key{HM}.pem"

%store home
%store certificate_file
%store public_key
%store private_key

mkdir: /Users/faermanj/aws_iot: File exists
Stored 'home' (str)
Stored 'certificate_file' (str)
Stored 'public_key' (str)
Stored 'private_key' (str)


In [5]:
certificateArn = ! aws iot create-keys-and-certificate \
  --set-as-active \
  --certificate-pem-outfile {certificate_file} \
  --public-key-outfile {public_key} \
  --private-key-outfile {private_key} \
  --query certificateArn \
  --output text
certificate_arn = certificateArn.s
certificate_arn

'arn:aws:iot:us-east-1:030555009967:cert/806259061a919fb48eb6d61a334e834babe8143e7c9bf6ee21572e0a734defd7'

Download the root Certification Authority used by AWS IoT


In [6]:
ca_file = f"{home}/aws_iot/ca{HM}.pem"
%store ca_file
! wget https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem -O {ca_file}

Stored 'ca_file' (str)
--2018-01-25 10:35:44--  https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem
Resolving www.symantec.com... 2.19.167.81, 2a02:26f0:13c:191::145b, 2a02:26f0:13c:185::145b
Connecting to www.symantec.com|2.19.167.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1758 (1.7K) [text/plain]
Saving to: ‘/Users/faermanj/aws_iot/ca1035.pem’

/Users/faermanj/aws 100%[===================>]   1.72K  --.-KB/s    in 0s      

2018-01-25 10:35:45 (29.9 MB/s) - ‘/Users/faermanj/aws_iot/ca1035.pem’ saved [1758/1758]



Double check that you have the 4 files required for TLS mutual authentication used by AWS IoT
1. Private Key
1. Public Key
1. Certificate
1. Root CA

In [7]:
! printf "Private Key {private_key} " && [ -f {private_key} ] && printf "found" || printf "not found"
! printf "\nPublic Key {public_key} " && [ -f {public_key} ] && printf "found" || printf "not found"
! printf "\nCertificate {certificate_file} " && [ -f {certificate_file} ] && printf "found" || printf "not found"
! printf "\nCertification Authority {ca_file} " && [ -f {ca_file} ] && printf "found" || printf "not found"

Private Key /Users/faermanj/aws_iot/private-key1035.pem found
Public Key /Users/faermanj/aws_iot/public-key1035.pem found
Certificate /Users/faermanj/aws_iot/certificate1035.pem found
Certification Authority /Users/faermanj/aws_iot/ca1035.pem found

Create an IAM authorization policy stating what is going to be allowed.

In [8]:
#TODO: Use a more restrictive policy
policy_document = '''{
  "Version": "2012-10-17",
  "Statement": [{
    "Effect": "Allow",
    "Action": "iot:*",
    "Resource": "*"}]
}'''

In [9]:
policy_name = f"NotebooksPolicy{HM}"
policyArn = ! aws iot create-policy \
    --policy-name {policy_name} \
    --policy-document '{policy_document}' \
    --query policyArn \
    --output text
policyArn.s

'arn:aws:iot:us-east-1:030555009967:policy/NotebooksPolicy1035'

Attach certificate to thing and policy

In [10]:
! aws iot attach-policy \
    --policy-name {policy_name} \
    --target {certificate_arn}

In [11]:
! aws iot attach-thing-principal \
  --thing-name {thing_name} \
  --principal {certificate_arn}

# Connecting to AWS IoT

AWS IoT provides account-specific endpoints for your things to connect:

In [12]:
endpoint_address = !aws iot describe-endpoint --query endpointAddress --output=text
endpoint_address = endpoint_address.s
%store endpoint_address
endpoint_address

Stored 'endpoint_address' (str)


'a3p88ooj9y6ruv.iot.us-east-1.amazonaws.com'

Ready to go! Create your MQTT client instance:

In [13]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id=f"NotebookCient{HM}"
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(10)
mqtt

# Receiving and Sending Messages

Connect to the service to send and receive messages:

In [14]:
mqtt.connect()

True

Subscribe to a topic:

In [15]:
topic_name ="notebook-topic"
%store topic_name
subscribe_qos = 1
def custom_callback(client, userdata, message):
    print("\nReceived a new message: \n")
    print(message.payload.decode("utf-8") )
    print("\n--------------\n")
    
mqtt.subscribe(topic_name, subscribe_qos, custom_callback)

Stored 'topic_name' (str)


True

Publish to a topic:

In [16]:
import uuid
import json
import datetime 
import random
import time
def genPayload():
    names = ["Sarah Jane", "Rory Williams", "Amy Pond", "Jack Harkness", "Clara Oswald", "Martha Jones", "Rose Tyler"] 
    name = random.choice(names)
    id = str(uuid.uuid4())
    now = int(time.time()*1000)
    noise = round(40 * random.random());
    data = {
        "id" : id ,
        "name" : name,
        "sistolic" : 110 + noise,
        "diastolic" : 70 + noise,
        "time_stamp" : now
    }
    payload = json.dumps(data)
    return payload
payloads = []
for x in range(100):
    payloads.append(genPayload())
%store payloads
payload = random.choice(payloads)
%store payload
payload

Stored 'payloads' (list)
Stored 'payload' (str)


'{"id": "0e40d97d-e11f-4609-9a5e-2afdaaf93e2f", "name": "Jack Harkness", "sistolic": 126, "diastolic": 86, "time_stamp": 1516872990737}'

In [17]:
publish_qos = 0
mqtt.publish(topic_name, payload , publish_qos)

True


Received a new message: 

{"id": "0e40d97d-e11f-4609-9a5e-2afdaaf93e2f", "name": "Jack Harkness", "sistolic": 126, "diastolic": 86, "time_stamp": 1516872990737}

--------------



The subscribe_qos and publish_qos arguments refer to the quality of service parameter of the MQTT protocol. Selecting the QoS level is a tradeoff between delivery guarantee and performance, see the following resources for more infomation:

https://www.hivemq.com/blog/mqtt-essentials-part-6-mqtt-quality-of-service-levels

https://docs.aws.amazon.com//iot/latest/developerguide/protocols.html

All working fine, done for now, disconnect.

In [18]:
mqtt.disconnect()

True